*Copyright (c) Microsoft Corporation. All rights reserved.*

*Licensed under the MIT License.*

# Text Classification of MultiNLI Sentences using BERT with Azure ML Pipelines

In [1]:
import sys
sys.path.append("../../")
import os
import json
import random
import shutil
import pandas as pd

from utils_nlp.models.bert.common import Language, Tokenizer
from utils_nlp.azureml import azureml_utils
from utils_nlp.dataset.multinli import get_generator

from sklearn.preprocessing import LabelEncoder
import azureml.core
from azureml.core import Datastore, Experiment,  get_run
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.core.compute import ComputeTarget,  AmlCompute
from azureml.exceptions import ComputeTargetException
from azureml.data.data_reference import DataReference
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.widgets import RunDetails
from azureml.train.dnn import PyTorch
from azureml.core.runconfig import MpiConfiguration
from azureml.pipeline.steps import EstimatorStep

print("System version: {}".format(sys.version))
print("Azure ML SDK Version:", azureml.core.VERSION)

System version: 3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]
Azure ML SDK Version: 1.0.48


## 0. Introduction

In this notebook, we fine-tune and evaluate a pretrained [BERT](https://arxiv.org/abs/1810.04805) model on a subset of the [MultiNLI](https://www.nyu.edu/projects/bowman/multinli/) dataset using [AzureML](https://azure.microsoft.com/en-us/services/machine-learning-service/) Pipelines.

We use a [distributed sequence classifier](../../utils_nlp/bert/sequence_classification_distributed.py) that wraps [Hugging Face's PyTorch implementation](https://github.com/huggingface/pytorch-pretrained-BERT) of Google's [BERT](https://github.com/google-research/bert).

The notebooks acts as a template to,
1. Process a massive dataset in parallel by dividing the dataset into chunks using [DASK](https://dask.org/) .
2. Perform distributed training on AzureML compute on these processed chunks.

In [2]:
LABEL_COL = "genre"
TEXT_COL = "sentence1"
DATA_FOLDER = "../../data/temp"
TRAIN_FOLDER = "../../data/temp/train"
TEST_FOLDER = "../../data/temp/test"
BERT_CACHE_DIR = "../../data/temp"
LANGUAGE = Language.ENGLISH
TO_LOWER = True
MAX_LEN = 150
BATCH_SIZE = 32
NUM_GPUS = 2
NUM_EPOCHS = 1
TRAIN_SIZE = 0.6
TEXT_COL = "sentence1"
ENCODED_LABEL_COL = "label"
TOKEN_COL = "tokens"
MASK_COL = "mask"
NUM_PARTITIONS = None
LABELS = ['telephone', 'government', 'travel', 'slate', 'fiction']
PROJECT_FOLDER = "../../"

In this example we will use AzureML pipelines to execute training pipelines. Each preprocessing step is included as a step in the pipeline. For a more detailed walkthrough of what pipelines are with a getting started guidelines check this [notebook](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-getting-started.ipynb). We start by doing some AzureML related setup below.

### 0.1 Create a workspace

First, go through the [Configuration](https://github.com/Azure/MachineLearningNotebooks/blob/master/configuration.ipynb) notebook to install the Azure Machine Learning Python SDK and create an Azure ML `Workspace`. This will create a config.json file containing the values needed below to create a workspace.

**Note**: you do not need to fill in these values if you have a config.json in the same folder as this notebook

In [3]:
ws = azureml_utils.get_or_create_workspace(
    subscription_id="<SUBSCRIPTION_ID>",
    resource_group="<RESOURCE_GROUP>",
    workspace_name="<WORKSPACE_NAME>",
    workspace_region="<WORKSPACE_REGION>",
)

Performing interactive authentication. Please follow the instructions on the terminal.


WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
WARNING - You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.


### 0.2 Create a compute target

In [4]:
# choose your cluster
cluster_name = "pipelines-tc-12"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print("Found existing compute target.")
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="STANDARD_NC12", max_nodes=8
    )

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current AmlCompute.
print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 2, 'targetNodeCount': 2, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 2, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2019-07-31T03:37:04.620000+00:00', 'errors': None, 'creationTime': '2019-07-25T04:16:20.598768+00:00', 'modifiedTime': '2019-07-25T04:16:36.486727+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 2, 'maxNodeCount': 10, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC12'}


## 1. Preprocessing

The pipeline is defined by a series of steps, the first being a PythonScriptStep which utilizes [DASK](https://dask.org/) to load dataframes in partitions allowing us to load and preprocess different sets of data in parallel.

### 1.1 Read Dataset

In [5]:
train_batches = get_generator(DATA_FOLDER, "train", num_batches=NUM_PARTITIONS, batch_size=10e6)
test_batches = get_generator(DATA_FOLDER, "dev_matched", num_batches=NUM_PARTITIONS, batch_size=10e6)

### 1.2 Preprocess and Tokenize

In the classification task, we use the first sentence only as the text input, and the corresponding genre as the label. Select the examples corresponding to one of the entailment labels (*neutral* in this case) to avoid duplicate rows, as the sentences are not unique, whereas the sentence pairs are.

Once filtered, we encode the labels. To do this, fit a label encoder with the known labels in a MNLI dataset.

In [6]:
os.makedirs(TRAIN_FOLDER, exist_ok=True)
os.makedirs(TEST_FOLDER, exist_ok=True)

labels = LABELS
label_encoder = LabelEncoder()
label_encoder.fit(labels)

num_train_partitions = 0
for batch in train_batches:
    batch = batch[batch["gold_label"]=="neutral"]
    batch[ENCODED_LABEL_COL] = label_encoder.transform(batch[LABEL_COL])
    batch.to_csv(TRAIN_FOLDER+"/batch{}.csv".format(str(num_train_partitions)))
    num_train_partitions += 1
    
num_test_partitions = 0
for batch in test_batches:
    batch = batch[batch["gold_label"]=="neutral"]
    batch[ENCODED_LABEL_COL] = label_encoder.transform(batch[LABEL_COL])
    batch.to_csv(TEST_FOLDER+"/batch{}.csv".format(str(num_test_partitions)))
    num_test_partitions += 1

Once we have the partitions of data ready they are uploaded to the datastore.

In [ ]:
ds = ws.get_default_datastore()
ds.upload(src_dir=TRAIN_FOLDER, target_path="mnli_data/train", overwrite=True, show_progress=False)
ds.upload(src_dir=TEST_FOLDER, target_path="mnli_data/test", overwrite=True, show_progress=False)

In [7]:
shutil.rmtree(TRAIN_FOLDER)
shutil.rmtree(TEST_FOLDER)

We can now parallely operate on each batch to tokenize the data and preprocess the tokens. To do this, we create a PythonScript step below.

In [30]:
%%writefile preprocess.py
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.
import argparse
import logging
import os

import pandas as pd

from utils_nlp.models.bert.common import Language, Tokenizer

LABEL_COL = "genre"
TEXT_COL = "sentence1"
LANGUAGE = Language.ENGLISH
TO_LOWER = True
MAX_LEN = 150

logger = logging.getLogger(__name__)


def tokenize(df):
    """Tokenize the text documents and convert them to lists of tokens using the BERT tokenizer.
    Args:
        df(pd.Dataframe): Dataframe with training or test samples

    Returns:

        list: List of lists of tokens for train set.

    """
    tokenizer = Tokenizer(
        LANGUAGE, to_lower=TO_LOWER)
    tokens = tokenizer.tokenize(list(df[TEXT_COL]))

    return tokens


def preprocess(tokens):
    """ Preprocess method that does the following,
            Convert the tokens into token indices corresponding to the BERT tokenizer's vocabulary
            Add the special tokens [CLS] and [SEP] to mark the beginning and end of a sentence
            Pad or truncate the token lists to the specified max length
            Return mask lists that indicate paddings' positions
            Return token type id lists that indicate which sentence the tokens belong to (not needed
            for one-sequence classification)

    Args:
        tokens(pd.Dataframe): Dataframe with tokens for train set.

    Returns:
        list: List of lists of tokens for train or test set with special tokens added.
        list: Input mask.
    """
    tokenizer = Tokenizer(
        LANGUAGE, to_lower=TO_LOWER)
    tokens, mask, _ = tokenizer.preprocess_classification_tokens(
        tokens, MAX_LEN
    )

    return tokens, mask


parser = argparse.ArgumentParser()
parser.add_argument("--input_data", type=str, help="input data")
parser.add_argument("--output_data", type=str, help="Path to the output file.")

args = parser.parse_args()
input_data = args.input_data
output_data = args.output_data
output_dir = os.path.dirname(os.path.abspath(output_data))

if output_dir is not None:
    os.makedirs(output_dir, exist_ok=True)
    logger.info("%s created" % output_dir)

df = pd.read_csv(args.input_data)
tokens_array = tokenize(df)
tokens_array, mask_array = preprocess(tokens_array)

df['tokens'] = tokens_array
df['mask'] = mask_array

# Filter columns
cols = ['tokens', 'mask', 'label']
df = df[cols]
df.to_csv(output_data, header=False, index=False)
logger.info("Completed")

Writing preprocess.py


In [31]:
preprocess_file = os.path.join(PROJECT_FOLDER,'utils_nlp/models/bert/preprocess.py')
shutil.move('preprocess.py',preprocess_file)

'../../utils_nlp/models/bert/preprocess.py'

Create a conda environment for the steps below.

In [32]:
conda_dependencies = CondaDependencies.create(
    conda_packages=[
        "numpy",
        "scikit-learn",
        "pandas",
    ],
    pip_packages=["azureml-sdk==1.0.43.*", 
                  "torch==1.1", 
                  "tqdm==4.31.1",
                 "pytorch-pretrained-bert>=0.6"],
    python_version="3.6.8",
)
run_config = RunConfiguration(conda_dependencies=conda_dependencies)
run_config.environment.docker.enabled = True

Then create the list of steps that use the preprocess.py created above. We use the output of these steps as input to training in the next section.

In [33]:
processed_train_files = []
processed_test_files = []
ds = ws.get_default_datastore()

for i in range(num_train_partitions):
        input_data = DataReference(datastore=ds, 
                                   data_reference_name='train_batch_{}'.format(str(i)), 
                                   path_on_datastore='mnli_data/train/batch{}.csv'.format(str(i)),
                                   overwrite=False)

        output_data = PipelineData(name="train{}".format(str(i)), datastore=ds,
                       output_path_on_compute='mnli_data/processed_train/batch{}.csv'.format(str(i)))

        step = PythonScriptStep(
            name='preprocess_step_train_{}'.format(str(i)),
            arguments=["--input_data", input_data, "--output_data", output_data],
            script_name= 'utils_nlp/models/bert/preprocess.py',
            inputs=[input_data],
            outputs=[output_data],
            source_directory=PROJECT_FOLDER,
            compute_target=compute_target,
            runconfig=run_config,
            allow_reuse=False,
        )
        
        processed_train_files.append(output_data)         
            
for i in range(num_test_partitions):
            input_data = DataReference(datastore=ds, 
                                       data_reference_name='test_batch_{}'.format(str(i)), 
                                       path_on_datastore='mnli_data/test/batch{}.csv'.format(str(i)),
                                       overwrite=False)
        
            output_data = PipelineData(name="test{}".format(str(i)), datastore=ds,
                        output_path_on_compute='mnli_data/processed_test/batch{}.csv'.format(str(i)))
            
            step = PythonScriptStep(
                name='preprocess_step_test_{}'.format(str(i)),
                arguments=["--input_data", input_data, "--output_data", output_data],
                script_name= 'utils_nlp/models/bert/preprocess.py',
                inputs=[input_data],
                outputs=[output_data],
                source_directory=PROJECT_FOLDER,
                compute_target=compute_target,
                runconfig=run_config,
                allow_reuse=False,
            )
            
            processed_test_files.append(output_data)

## 2. Train and Score

Once the data is processed and available on datastore, we  train the classifier using the training examples. This involves fine-tuning the BERT Transformer and learning a linear classification layer on top of that. After training is complete we score the performance of the model on the test dataset

The training is distributed and is done AzureML's capability to support distributed using MPI with horovod. 


### 2.1 Setup training script

In [34]:
%%writefile train.py
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT License.

import argparse
import json
import logging
import os
import torch
from sklearn.metrics import classification_report

from utils_nlp.models.bert.common import Language
from utils_nlp.models.bert.sequence_classification_distributed import (
    BERTSequenceDistClassifier,
)
from utils_nlp.common.timer import Timer

BATCH_SIZE = 32
NUM_GPUS = 2
NUM_EPOCHS = 1
LABELS = ["telephone", "government", "travel", "slate", "fiction"]
OUTPUT_DIR = "./outputs/"

logger = logging.getLogger(__name__)

parser = argparse.ArgumentParser()
parser.add_argument(
    "--train_files",
    nargs="+",
    default=[],
    help="List of file paths to all the files in train dataset.",
)

parser.add_argument(
    "--test_files",
    nargs="+",
    default=[],
    help="List of file paths to all the files in test dataset.",
)

args = parser.parse_args()
train_files = [file.strip() for file in args.train_files]
test_files = [file.strip() for file in args.test_files]

# Handle square brackets from train list
train_files[0] = train_files[0][1:]
train_files[len(train_files) - 1] = train_files[len(train_files) - 1][:-1]

# Handle square brackets from test list
test_files[0] = test_files[0][1:]
test_files[len(test_files) - 1] = test_files[len(test_files) - 1][:-1]

# Train
classifier = BERTSequenceDistClassifier(
    language=Language.ENGLISH, num_labels=len(LABELS)
)
with Timer() as t:
    classifier.fit(
        train_files,
        num_gpus=NUM_GPUS,
        num_epochs=NUM_EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=True,
    )

# Predict
preds, labels_test = classifier.predict(
    test_files, num_gpus=NUM_GPUS, batch_size=BATCH_SIZE
)

results = classification_report(
    labels_test, preds, target_names=LABELS, output_dict=True
)

# Write out results.
result_file = os.path.join(OUTPUT_DIR, "results.json")
with open(result_file, "w+") as fp:
    json.dump(results, fp)

# Save model
model_file = os.path.join(OUTPUT_DIR, "model.pt")
torch.save(classifier.model.state_dict(), model_file)

Writing train.py


In [35]:
train_file = os.path.join(PROJECT_FOLDER,'utils_nlp/models/bert/train.py')
shutil.move('train.py',train_file)

'../../utils_nlp/models/bert/train.py'

### 2.2 Create a Pytorch Estimator

We create a Pytorch Estimator using AzureML SDK and additonally define an EstimatorStep to run it on AzureML pipelines.

In [36]:
estimator = PyTorch(source_directory=PROJECT_FOLDER,
                    compute_target=compute_target,
                    entry_script='utils_nlp/models/bert/train.py',
                    node_count=4,
                    distributed_training=MpiConfiguration(),
                    process_count_per_node=2,
                    use_gpu=True,
                    conda_packages=['scikit-learn=0.20.3', 'numpy>=1.16.0', 'pandas'],
                    pip_packages=["tqdm==4.31.1","pytorch-pretrained-bert>=0.6"]
                   )

WARNING - framework_version is not specified, defaulting to version 1.1.
WARNING - 'process_count_per_node' parameter will be deprecated. Please use it as part of 'distributed_training' parameter.


In [37]:
inputs = processed_train_files + processed_test_files

est_step = EstimatorStep(name="Estimator-Train", 
                         estimator=estimator, 
                         estimator_entry_script_arguments=[
                             '--train_files',  str(processed_train_files),
                             '--test_files', str(processed_test_files)],
                         inputs = inputs,
                         runconfig_pipeline_params=None, 
                         compute_target=compute_target)

### 2.3 Submit the pipeline

The model is fine tuned on AML Compute and takes ~45 minutes to train.

In [38]:
pipeline = Pipeline(workspace=ws, steps=[est_step])
experiment = Experiment(ws, 'NLP-TC-BERT-distributed')
pipeline_run = experiment.submit(pipeline)

Created step Estimator-Train [0fc7c0e5][55d62442-870b-47ef-8d63-0c7630e7c111], (This step will run and generate new outputs)
Created step preprocess_step_train_0 [8ec688c6][0ee5c911-1583-4e6f-bf4a-0d4521ecc47f], (This step will run and generate new outputs)
Created step preprocess_step_train_1 [0863e051][f11ec98d-05c3-44ac-841c-d953e624e396], (This step will run and generate new outputs)
Created step preprocess_step_train_2 [af6586f3][3c07d2a2-ba11-4c87-9ab6-826ec3fe3dd2], (This step will run and generate new outputs)
Created step preprocess_step_train_3 [3a1e8838][ade4b1e6-e39b-4773-8cf2-753b99461b06], (This step will run and generate new outputs)
Created step preprocess_step_train_4 [a9cb96ca][1f2a11f1-f4b5-4538-b68f-7b55f3a84f4b], (This step will run and generate new outputs)
Created step preprocess_step_train_5 [c1ba441a][d62b3d14-27ce-485f-a1d3-a61067b54598], (This step will run and generate new outputs)
Created step preprocess_step_train_6 [cffd3803][ccf45717-03cc-41ac-ab21-183a3

Using data reference train_batch_7 for StepId [a422ef06][d2cb8ea4-4650-4143-85cc-3a50b2d92fb3], (Consumers of this data are eligible to reuse prior runs.)
Using data reference train_batch_8 for StepId [9648bacd][0b8ae803-5f49-4361-9c27-156a4f69486b], (Consumers of this data are eligible to reuse prior runs.)
Using data reference train_batch_9 for StepId [c57af85a][2d50bd14-2fb0-4742-8805-3bd1dc19c3c3], (Consumers of this data are eligible to reuse prior runs.)
Using data reference train_batch_10 for StepId [f6b5653a][cba68341-2c4a-432e-a4bc-38af276684a6], (Consumers of this data are eligible to reuse prior runs.)
Using data reference train_batch_11 for StepId [8a1ff5e1][f7250703-3bad-4db6-9c4a-e68c98565f0a], (Consumers of this data are eligible to reuse prior runs.)
Using data reference train_batch_12 for StepId [5fdf1ab0][1e833559-7b27-422d-ad18-c7b8421af99a], (Consumers of this data are eligible to reuse prior runs.)
Using data reference train_batch_13 for StepId [fda0e060][c28e0d83-

In [39]:
RunDetails(pipeline_run).show()

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

In [29]:
#If you would like to cancel the job for any reasons uncomment the code below.
#pipeline_run.cancel()

In [40]:
#wait for the run to complete before continuing in the notebook
pipeline_run.wait_for_completion()

PipelineRunId: e577424c-005c-48a0-a7b5-98ddb8145533
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/nlprg/providers/Microsoft.MachineLearningServices/workspaces/MAIDAPTest/experiments/NLP-TC-BERT-distributed/runs/e577424c-005c-48a0-a7b5-98ddb8145533
PipelineRun Status: Running


StepRunId: 17799016-02a8-45f9-9c0b-b47cb7c6716a
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/nlprg/providers/Microsoft.MachineLearningServices/workspaces/MAIDAPTest/experiments/NLP-TC-BERT-distributed/runs/17799016-02a8-45f9-9c0b-b47cb7c6716a
StepRun( Estimator-Train ) Status: Running

Streaming azureml-logs/70_driver_log_rank_0.txt

100%|██████████| 407873900/407873900 [00:08<00:00, 46186846.69B/s]
t_total value of -1 results in schedule not being applied
t_total value of -1 results in schedule not being applied
80f8842b87664eeead53155d8300d04e000045:137:168 [0] NCCL IN

80f8842b87664eeead53155d8300d04e000045:137:168 [0] NCCL INFO Ring 01 : 0 -> 1 [send] via NET/Socket/0
80f8842b87664eeead53155d8300d04e000045:137:168 [0] NCCL INFO Using 128 threads, Min Comp Cap 3, Trees disabled
80f8842b87664eeead53155d8300d04e000045:137:168 [0] NCCL INFO comm 0x7f4ed0317620 rank 0 nranks 4 cudaDev 0 nvmlDev 0 - Init COMPLETE
80f8842b87664eeead53155d8300d04e000045:137:168 [0] NCCL INFO Launch mode Parallel
Train Epoch: 0 [0/32713 (0%)]	Loss: 1.593874
Train Epoch: 0 [13088/32713 (40%)]	Loss: 0.271189
Train Epoch: 0 [26176/32713 (80%)]	Loss: 0.192379

 98%|█████████▊| 96/98 [00:31<00:00,  3.59it/s]
128it [00:45,  3.07it/s]                       


The experiment completed successfully. Finalizing run...
Cleaning up all outstanding Run operations, waiting 300.0 seconds
1 items cleaning up...
Cleanup took 0.00037097930908203125 seconds

StepRun(Estimator-Train) Execution Summary
StepRun( Estimator-Train ) Status: Finished

Warnings:
Failed to upload AzureMLCompute logs to

{'runId': '17799016-02a8-45f9-9c0b-b47cb7c6716a', 'target': 'pipelines-tc-12', 'status': 'Completed', 'startTimeUtc': '2019-07-31T05:06:22.597622Z', 'endTimeUtc': '2019-07-31T05:59:51.446093Z', 'warnings': [{'message': 'Failed to upload AzureMLCompute logs to RunHistory. The logs can be accessed directly on the file share under 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/nlprg/workspaces/maidaptest/experiments/azureml/jobs/17799016-02a8-45f9-9c0b-b47cb7c6716a/b027822b-f0b4-4585-91d8-6fe54fe804a4.'}], 'properties': {'azureml.runsource': 'azureml.StepRun', 'ContentSnapshotId': 'fcf56a80-d0c9-4e84-9f7e-ad8ad11e11a2', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.pipelinerunid': 'e577424c-005c-48a0-a7b5-98ddb8145533', '_azureml.ComputeTargetType': 'batchai', 'AzureML.DerivedImageName': 'azureml/azureml_150cd4ca02e9b5ae84d7bbbededff028'}, 'runDefinition': {'script': 'utils_nlp/models/bert/train.py', 'arguments': ['--train_files', '[$AZUREML_DATAREFERENCE_train0', ' $A





StepRunId: ba50d0db-bd5c-42e7-8786-c984048aaaf1
Link to Portal: https://mlworkspace.azure.ai/portal/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/nlprg/providers/Microsoft.MachineLearningServices/workspaces/MAIDAPTest/experiments/NLP-TC-BERT-distributed/runs/ba50d0db-bd5c-42e7-8786-c984048aaaf1

StepRun(preprocess_step_train_47) Execution Summary
StepRun( preprocess_step_train_47 ) Status: Finished
{'runId': 'ba50d0db-bd5c-42e7-8786-c984048aaaf1', 'target': 'pipelines-tc-12', 'status': 'Completed', 'startTimeUtc': '2019-07-31T05:03:41.662908Z', 'endTimeUtc': '2019-07-31T05:05:56.066053Z', 'properties': {'azureml.runsource': 'azureml.StepRun', 'ContentSnapshotId': 'fcf56a80-d0c9-4e84-9f7e-ad8ad11e11a2', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.pipelinerunid': 'e577424c-005c-48a0-a7b5-98ddb8145533', '_azureml.ComputeTargetType': 'batchai', 'AzureML.DerivedImageName': 'azureml/azureml_ead8341f2d197d2c755041a285368248'}, 'runDefini

{'runId': '9f508b5d-04d5-4cab-b786-b61fd951c85a', 'target': 'pipelines-tc-12', 'status': 'Completed', 'startTimeUtc': '2019-07-31T05:03:36.396019Z', 'endTimeUtc': '2019-07-31T05:05:30.355409Z', 'properties': {'azureml.runsource': 'azureml.StepRun', 'ContentSnapshotId': 'fcf56a80-d0c9-4e84-9f7e-ad8ad11e11a2', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.pipelinerunid': 'e577424c-005c-48a0-a7b5-98ddb8145533', '_azureml.ComputeTargetType': 'batchai', 'AzureML.DerivedImageName': 'azureml/azureml_ead8341f2d197d2c755041a285368248'}, 'runDefinition': {'script': 'utils_nlp/models/bert/preprocess.py', 'arguments': ['--input_data', '$AZUREML_DATAREFERENCE_train_batch_24', '--output_data', '$AZUREML_DATAREFERENCE_train24'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'pipelines-tc-12', 'dataReferences': {'train_batch_24': {'dataStoreName': 'liqungensen', 'mode': 'Mount', 'pathOnDataStore': 'mnli_data/train/batch24.csv',

...

{'runId': '8ed16974-1994-4816-84a8-abf7e70663f5', 'target': 'pipelines-tc-12', 'status': 'Completed', 'startTimeUtc': '2019-07-31T04:49:53.983978Z', 'endTimeUtc': '2019-07-31T04:52:32.069919Z', 'properties': {'azureml.runsource': 'azureml.StepRun', 'ContentSnapshotId': 'fcf56a80-d0c9-4e84-9f7e-ad8ad11e11a2', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.pipelinerunid': 'e577424c-005c-48a0-a7b5-98ddb8145533', '_azureml.ComputeTargetType': 'batchai', 'AzureML.DerivedImageName': 'azureml/azureml_ead8341f2d197d2c755041a285368248'}, 'runDefinition': {'script': 'utils_nlp/models/bert/preprocess.py', 'arguments': ['--input_data', '$AZUREML_DATAREFERENCE_train_batch_13', '--output_data', '$AZUREML_DATAREFERENCE_train13'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'pipelines-tc-12', 'dataReferences': {'train_batch_13': {'dataStoreName': 'liqungensen', 'mode': 'Mount', 'pathOnDataStore': 'mnli_data/train/batch13.csv',

{'runId': 'd28e5807-82e1-4fe9-8b6b-89eaeb5e287b', 'target': 'pipelines-tc-12', 'status': 'Completed', 'startTimeUtc': '2019-07-31T04:49:53.703054Z', 'endTimeUtc': '2019-07-31T04:52:39.7458Z', 'properties': {'azureml.runsource': 'azureml.StepRun', 'ContentSnapshotId': 'fcf56a80-d0c9-4e84-9f7e-ad8ad11e11a2', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.pipelinerunid': 'e577424c-005c-48a0-a7b5-98ddb8145533', '_azureml.ComputeTargetType': 'batchai', 'AzureML.DerivedImageName': 'azureml/azureml_ead8341f2d197d2c755041a285368248'}, 'runDefinition': {'script': 'utils_nlp/models/bert/preprocess.py', 'arguments': ['--input_data', '$AZUREML_DATAREFERENCE_train_batch_36', '--output_data', '$AZUREML_DATAREFERENCE_train36'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'pipelines-tc-12', 'dataReferences': {'train_batch_36': {'dataStoreName': 'liqungensen', 'mode': 'Mount', 'pathOnDataStore': 'mnli_data/train/batch36.csv', '

{'runId': '4c60a350-3223-4fb7-a4aa-552acc3146bf', 'target': 'pipelines-tc-12', 'status': 'Completed', 'startTimeUtc': '2019-07-31T04:46:06.268381Z', 'endTimeUtc': '2019-07-31T04:50:20.50754Z', 'properties': {'azureml.runsource': 'azureml.StepRun', 'ContentSnapshotId': 'fcf56a80-d0c9-4e84-9f7e-ad8ad11e11a2', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.pipelinerunid': 'e577424c-005c-48a0-a7b5-98ddb8145533', '_azureml.ComputeTargetType': 'batchai', 'AzureML.DerivedImageName': 'azureml/azureml_ead8341f2d197d2c755041a285368248'}, 'runDefinition': {'script': 'utils_nlp/models/bert/preprocess.py', 'arguments': ['--input_data', '$AZUREML_DATAREFERENCE_train_batch_41', '--output_data', '$AZUREML_DATAREFERENCE_train41'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'pipelines-tc-12', 'dataReferences': {'train_batch_41': {'dataStoreName': 'liqungensen', 'mode': 'Mount', 'pathOnDataStore': 'mnli_data/train/batch41.csv', 

{'runId': '5cc8a1a0-36eb-4017-86a9-c48a098a2f9e', 'target': 'pipelines-tc-12', 'status': 'Completed', 'startTimeUtc': '2019-07-31T04:46:06.260771Z', 'endTimeUtc': '2019-07-31T04:50:20.774584Z', 'properties': {'azureml.runsource': 'azureml.StepRun', 'ContentSnapshotId': 'fcf56a80-d0c9-4e84-9f7e-ad8ad11e11a2', 'StepType': 'PythonScriptStep', 'ComputeTargetType': 'AmlCompute', 'azureml.pipelinerunid': 'e577424c-005c-48a0-a7b5-98ddb8145533', '_azureml.ComputeTargetType': 'batchai', 'AzureML.DerivedImageName': 'azureml/azureml_ead8341f2d197d2c755041a285368248'}, 'runDefinition': {'script': 'utils_nlp/models/bert/preprocess.py', 'arguments': ['--input_data', '$AZUREML_DATAREFERENCE_train_batch_9', '--output_data', '$AZUREML_DATAREFERENCE_train9'], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'pipelines-tc-12', 'dataReferences': {'train_batch_9': {'dataStoreName': 'liqungensen', 'mode': 'Mount', 'pathOnDataStore': 'mnli_data/train/batch9.csv', 'pa

'Finished'

### 2.4 Download and analyze results

In [ ]:
step_run = pipeline_run.find_step_run("Estimator-Train")[0]
file_names = ['outputs/results.json', 'outputs/model.pt']
azureml_utils.get_output_files(step_run, './outputs', file_names=file_names)

In [ ]:
with open('outputs/results.json', 'r') as handle:
    parsed = json.load(handle)
    print(pd.DataFrame.from_dict(parsed).transpose())

Finally clean up any intermediate files we created.

In [ ]:
os.remove(train_file)
os.remove(preprocess_file)